In [1]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")

/Users/peerasit/senior_project/STELLA-Backend/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Prepare PDF File

In [48]:
# # import pymupdf
# # from langchain_core.documents import Document
# # doc = pymupdf.open('../../chunking/pdfs/bts_esg_2023.pdf')

# # import sys
# # # sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend")
# # from chunking.esg_file import extractAll
# # doc = extractAll(file_name="../../chunking/pdfs/bts_esg_2023.pdf")

import pdfplumber

doc = []
with pdfplumber.open("../../chunking/pdfs/bts_esg_2023.pdf") as pdf:
    page_size = len(pdf.pages)
    doc = [i.extract_text() for i in pdf.pages]

#### Recursive Chunking

In [52]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=500,
    # length_function=len,
    is_separator_regex=False,
)

text_chunks = text_splitter.create_documents(doc)
# text_chunks = text_splitter.create_documents([d.get_text() for d in doc])

In [53]:
# len(text_chunks)

#### Semantic Chunking

In [54]:
# import pymupdf
# from langchain_core.documents import Document

# semantic_chunker = SemanticChunker(embeddings=embedding_model, breakpoint_threshold_type="percentile")
# # text_chunks = semantic_chunker.create_documents([d.get_text() for d in doc])
# text_chunks = semantic_chunker.create_documents(doc)

#### Hybrid Chunking

In [55]:
# import sys
# sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend/")
# from  chunking.esg_file import esgFileChunking
# text_chunks = (esgFileChunking(file_path="../../chunking/pdfs/bts_esg_2023.pdf"))
# # from chunking.ndc_file import ndcFileChunking

# # text_chunks = ndcFileChunking(content="../../chunking/pdfs/general/Thailand_INDCs_2015.pdf", file_name="Thailand_INDCs_2015.pdf")

## Setup STELLA CORE DB

In [56]:
import sys
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend")
sys.path.insert(0, "/Users/peerasit/senior_project/STELLA-Backend/milvus")

from milvus.core import Core
from milvus.schema import INDEX_PARAMS, DATA_SOURCE_SCHEMA

core = Core(
            database_name="esg_evaluate",
            schema=DATA_SOURCE_SCHEMA,
            dense_embedding_model=embedding_model,
            create_first_node=True,
            system_prune_first_node=True,
            token=""
        )

[CORE] Initializing Milvus Database Core...
[DB] init Embedding Model...
[DB] init Embedding Model Successfully.
[DB] Found Database: esg_evaluate
[DB] Found Collection "cnode_1".
[DB] Drop Collection "cnode_1"...
cnode_1 has: 0 entities
[DB] Drop Collection "cnode_1" Successfully.
[DB] Create Collection "cnode_1"
[DB] Collection "cnode_1" Is Ready.
[DB] Found Collection "gnode_1".
[DB] Drop Collection "gnode_1"...
gnode_1 has: 69 entities
[DB] Drop Collection "gnode_1" Successfully.
[DB] Create Collection "gnode_1"
[DB] Collection "gnode_1" Is Ready.
[DB] Found Collection "frontend_query_general_documents".
[DB] Drop Collection "frontend_query_general_documents"...
frontend_query_general_documents has: 1 entities
[DB] Drop Collection "frontend_query_general_documents" Successfully.
[DB] Create Collection "frontend_query_general_documents"
[DB] Collection "frontend_query_general_documents" Is Ready.
Create Schma Successfuly.


## Add Document 

In [57]:
core.add_document(name="esg", documents=text_chunks, node_type="g", description="this is esg file")

<Collection>:
-------------
<name>: gnode_1
<description>: Schema for Data Source Collection
<schema>: {'auto_id': True, 'description': 'Schema for Data Source Collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'dense_vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 1024}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 8192}}, {'name': 'metadata', 'description': '', 'type': <DataType.JSON: 23>}], 'enable_dynamic_field': True}

<class 'pymilvus.orm.collection.Collection'>
[DB] Insert Query FrontEnd Successfuly.
[DB] Create New Partition
[DB] Partition esg: 71 entities


## Setup Retrieval DB

In [58]:
config ={
    "k": 4,
    "partition_names": ["esg"],
}
chunk_retriver = core.initVectorStore(collection_name="gnode_1", partition_names=["esg"], search_kwargs=config)

In [59]:
chunk_retriver.invoke("มาตราการ")

[Document(metadata={}, page_content='3.1 ขอบเขตการเปิดเผยการจัดการน้ำ\nประจำปี\nรายละเอียด\n2565 2566\nประเภทขอบเขต บริษัท -\nจำนวนขอบเขตทั้งหมด - -\nจำนวนที่เปิดเผย - -\n3.2 ปริมาณการใช้น้ำของบริษัท จำแนกตามแหล่งน้ำ\nประจำปี\nรายละเอียด หน่วย\n2564 2565 2566\nเป้าหมายของปริมาณการใช้น้ำ ลูกบาศก์เมตร 220,000.00 220,000.00 -\nปริมาณการใช้น้ำรวม ลูกบาศก์เมตร 208,800.00 199,500.00 -\nปริมาณการใช้น้ำประปาหรือน้ำจากองค์กรอื่น ลูกบาศก์เมตร 208,800.00 199,500.00 -\nปริมาณการใช้น้ำผิวดิน ลูกบาศก์เมตร - - -\nปริมาณการใช้น้ำบาดาล ลูกบาศก์เมตร - - -\nปริมาณการใช้น้ำทะเล ลูกบาศก์เมตร - - -\nปริมาณการใช้น้ำจากกระบวนการผลิต ลูกบาศก์เมตร - - -\nผลต่างระหว่างปริมาณการใช้น้ำรวมเทียบกับเป้าหมาย(*) ลูกบาศก์เมตร -11,200.00 -20,500.00 -\nร้อยละของผลต่างระหว่างปริมาณการใช้น้ำรวมเทียบกับ\n% -5.09 -9.32 -\nเป้าหมาย(*)\nลูกบาศก์เมตร /\nอัตราส่วนปริมาณการใช้น้ำรวมต่อจำนวนพนักงานรวม 38.98 33.93 -\nคน /\nปี\nลูกบาศก์เมตร /\nอัตราส่วนปริมาณการใช้น้ำรวมต่อรายได้รวม(**) - - -\nราย\nได้พันบาท\nคำอธิบายเพิ่มเติม:\n(*) 

## Setup LLM For Generate Synthesised Data

In [60]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_API_KEY")

### LLM RAG

In [61]:
from langchain_core.prompts import ChatPromptTemplate

rag_template = """\
Use the following context to answer the user's query. If you cannot answer, please respond with 'I don't know'.

User's Query:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

In [62]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0, max_tokens=4096)

recursive_rag_chain = (
    {"context" : chunk_retriver, "question" : RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [63]:
synthetic_data_chunks = text_chunks
synthetic_data_chunks_size = len(synthetic_data_chunks)

## LLM Question

In [64]:
question_prompt = """\
You are a teacher preparing a test. Please create a question that can be answered by referencing the following context.

Context:
{context}
"""

question_prompt = ChatPromptTemplate.from_template(question_prompt)
question_chain = question_prompt | llm | StrOutputParser()

## LLM Ground Truth

In [65]:
ground_truth_prompt = """\
Use the following context and question to answer this question using *only* the provided context.

Question:
{question}

Context:
{context}
"""

ground_truth_prompt = ChatPromptTemplate.from_template(ground_truth_prompt)
ground_truth_chain = ground_truth_prompt | llm | StrOutputParser()

## Create Synthesised Data

In [66]:
questions = []
ground_truths_recursive = []
contexts = []
answers = []

c = 0
for chunk in synthetic_data_chunks:
  # print(c)
  questions.append(question_chain.invoke({"context" : chunk.page_content}))
  # contexts.append(chunk.page_content)
  # ground_truths_recursive.append(ground_truth_chain.invoke({"question" : questions[-1], "context" : str(contexts[-1])}))
  ground_truths_recursive.append(ground_truth_chain.invoke({"question" : questions[-1], "context" : chunk.page_content}))
  

  contexts.append([d.page_content for d in chunk_retriver.get_relevant_documents(questions[-1])])
  answers.append(recursive_rag_chain.invoke(questions[-1]))
  # c += 1

In [67]:
# [d.page_content for d in chunk_retriver.get_relevant_documents("d")]

In [68]:
from datasets import load_dataset, Dataset

qagc_list = []

for question, answer, context, ground_truth in zip(questions, answers, contexts, ground_truths_recursive):
  # print("q",question)
  # print("g",ground_truth)
  # print("a",answer)
  # print("c", context)
  qagc_list.append({
      "question" : question,
      "answer" : answer,
      "contexts" : context,
      "ground_truth" : ground_truth
  })

eval_dataset = Dataset.from_list(qagc_list)
eval_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 71
})

## Evaluation with RAGAS

In [69]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)

from ragas import evaluate

result = evaluate(
    eval_dataset,
    metrics=[
        # retrieval
        context_precision,
        context_recall,
    # generation
        # faithfulness,
        # answer_relevancy,
    ],
)
result

Evaluating: 100%|██████████| 142/142 [02:42<00:00,  1.14s/it]


{'context_precision': 0.7371, 'context_recall': 0.8303}

In [70]:
results_df = result.to_pandas()
results_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall
0,Question: Does BTS Group Limited have an Envir...,[ข้อมูลการดำเนินงานด้าน ESG\nบริษัท: บริษัท บี...,"Yes, BTS Group Limited has an Environmental Ma...","Yes, BTS Group Limited has an Environmental Ma...",0.750000,1.0
1,"Based on the context provided, please answer t...",[ร้อยละของผลต่างระหว่างปริมาณการใช้ไฟฟ้ารวมเที...,I don't know.,The difference between the total electricity c...,1.000000,1.0
2,"Based on the context provided, please calculat...",[ร้อยละของผลต่างระหว่างปริมาณการใช้ไฟฟ้ารวมเที...,I don't know.,To calculate the percentage difference between...,0.000000,1.0
3,What is the percentage of total electricity ex...,[ร้อยละของค่าใช้จ่ายการใช้ไฟฟ้ารวมต่อรายได้รวม...,I don't know.,To calculate the percentage of total electrici...,0.000000,1.0
4,"Based on the context provided, please answer t...","[ก๊าซธรรมชาติ ลูกบาศก์ฟุต 24,829.82 29,287.78 ...",I don't know.,"Based on the context provided, the total amoun...",1.000000,1.0
...,...,...,...,...,...,...
66,Question: What is the annual meeting fee for M...,[3.2 การเข้าร่วมประชุมของคณะกรรมการบริษัท\nจำน...,I don't know.,The annual meeting fee for Mr. Surapong Leohay...,0.000000,0.0
67,Question: What is the total compensation of th...,"[รวม - 340,000.00 6,564,000.00 -\n3.4.3 ค่าตอบ...",Total compensation of the company's board of d...,The total compensation of the company's board ...,0.916667,1.0
68,"Based on the context provided, please answer t...",[3.4.5 ค่าตอบแทนรวมของผู้บริหาร\nประจำปี\nรายล...,I don't know.,"No, the Employee Joint Investment Program (EJI...",1.000000,1.0
69,Question: Which individuals are listed as memb...,[(กรรมการ)\n6. นาย กวิน\n- 8/8 100.00 เข้าร่วม...,Members of the committee listed in the given c...,Members of the committee listed in the given c...,1.000000,0.7


In [71]:
# for i in results_df["retrieved_contexts"]:
#     print(len(i))

In [72]:
results_df['context_precision'].mean()

0.737089201830125

In [73]:
results_df['context_precision'].mean()

0.737089201830125